In [ ]:
!pip install virtualenv
!mkdir my_virtualenv
%cd my_virtualenv
!virtualenv my_env
!source my_env/bin/activate

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
!pip install langchain-groq

In [ ]:
!pip install sentence-transformers

In [3]:
from langchain_openai import ChatOpenAI
import os
from crewai_tools import PDFSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai_tools  import tool
from crewai import Crew
from crewai import Task
from crewai import Agent
from google.colab import userdata

In [4]:
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [5]:
llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=os.environ['GROQ_API_KEY'],
    model_name="llama3-8b-8192",
    temperature=0.1,
    max_tokens=1000,
)

In [7]:
!wget "https://www.icsi.edu/media/webmodules/CSJ/January/17.pdf"

--2024-06-24 19:50:14--  https://www.icsi.edu/media/webmodules/CSJ/January/17.pdf
Resolving www.icsi.edu (www.icsi.edu)... 103.109.6.133
Connecting to www.icsi.edu (www.icsi.edu)|103.109.6.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 871378 (851K) [application/pdf]
Saving to: ‘17.pdf’

17.pdf              100%[===================>] 850.96K   718KB/s    in 1.2s    

2024-06-24 19:50:17 (718 KB/s) - ‘17.pdf’ saved [871378/871378]



In [ ]:
!pip install langchain_huggingface

In [10]:
rag_tool = PDFSearchTool(pdf='/content/17.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Inserting batches in chromadb: 100%|██████████| 1/1 [00:10<00:00, 10.35s/it]


In [11]:
rag_tool.run("How does exercise price determine for ESOP?")


Using Tool: Search a PDF's content


'Relevant Content:\noptions or shares. The employees are only able to obtain these shares once the ESOP vesting term has completed. Exercise period: It means the time period which starts after the completion of vesting period within which an employee can exercise his/her right to apply for shares against the vested options in pursuance of the scheme of ESOP approved by the shareholders in general meeting by way of Special Resolution. Exercise price: Means the price payable by an employee for exercising the options granted in pursuance of the scheme of ESOP. ARTICLE JANUARY 2024 | 107 CHARTERED SECRETARY\n\nESOP? A. Companies are free to decide the exercise price, which may be issued at a discount or premium but the exercise price determined by the Company shall not be less than the par value of the shares. Q. Does it mandatory for the Company to issue and allot only fresh shares under ESOP scheme? A. No, it is not mandatory for the company to issue and allot only fresh shares under ESO

In [12]:
from crewai_tools  import tool
@tool
def router_tool(question):
  """Router Function"""
  if 'ESOP' in question:
    return 'vectorstore'
  else:
    return 'web_search'

In [13]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
web_search_tool = TavilySearchResults(k=3)

In [14]:
web_search_tool.run("How does exercise price determine for ESOP?")

[{'url': 'https://www.trica.co/equity/blog/how-to-calculate-exercise-price-in-esops/',
  'content': "To begin with, the exercise price in ESOP is used to determine the amount required to exercise the options and the tax ramifications of doing so. In return for the shares, the employee must pay the strike price multiplied by the number of vested options the employee desires to exercise. The difference between the stock's current Fair Market ..."},
 {'url': 'https://www.vega-equity.com/blog/how-to-calculate-exercise-price-in-esops',
  'content': "Now coming back to the exercise price in ESOPs, it is the price at which the shareholder can purchase the share options. Often termed the Strike price, the exercise price becomes active once the vesting period is completed. This price is set at the stock's fair market value at the time the stock options are granted."},
 {'url': 'https://www.fairvalueacademy.org/employee-share-options-esops/',
  'content': "The exercise multiple factors in early 

In [15]:
Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concepta related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [16]:
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [17]:
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)


In [18]:
@tool
def retriever_tool(context):
  """Based on the response from the router task use rag_tool or web_search_tool
     to retrieve answer for the question asked
  """
  if context == "websearch":
    return web_search_tool
  else:
    return rag_tool

In [19]:
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
   #tools=[retriever_tool],
)

In [20]:
Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [21]:
grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [22]:
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)


In [23]:
hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

In [24]:
@tool
def answer_grader_tool(context,question):
  "answer grader tool"
  if context == 'yes' or context == "YES":
    pass
  else:
    return web_search_tool

In [25]:
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [26]:
answer_task = Task(
    description=("Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'websearch' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,
    #tools=[answer_grader_tool],
)

In [27]:
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,

)

In [28]:
rag_crew = Crew(
    agents=[Router_Agent,Retriever_Agent,Grader_agent,hallucination_grader],
    tasks=[router_task,retriever_task,grader_task,hallucination_task],
    verbose=True,

)

In [29]:
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,

)

In [30]:
#inputs ={"question":"What is an ESOP?"}
inputs ={"question":"Does the ESOP supplement the salary of an employee?"}

In [31]:
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question Does the ESOP supplement the salary of an employee?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question is about ESOP, salary, and employee, which seems to be related to finance and employment concepts.
Action: router_tool
Action Input: {"question": "Does the ESOP supplement the salary of an employee?"} 

vectorstore

Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question Does the ESOP supplement the salary of an e

In [32]:
print(result)

Yes, an Employee Stock Ownership Plan (ESOP) can supplement the salary of an employee. An ESOP is a type of retirement plan that allows employees to own company stock, and the company can contribute a portion of the employee's salary to the plan. This can provide an additional source of income for the employee, supplementing their regular salary.


In [33]:
rag_crew = Crew(
    agents=[Router_Agent],
    tasks=[router_task],
    verbose=True,

)
#inputs ={"question":"What is an ESOP?"}
inputs ={"question":"Does the ESOP supplement the salary of an employee?"}
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question Does the ESOP supplement the salary of an employee?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question is about ESOP, salary, and employee, which seems to be related to finance and employment concepts.
Action: router_tool
Action Input: {"question": "Does the ESOP supplement the salary of an employee?"} 

vectorstore

Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore




In [34]:
result

'vectorstore'

In [35]:
rag_crew = Crew(
    agents=[Router_Agent,Retriever_Agent],
    tasks=[router_task,retriever_task],
    verbose=True,

)
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question Does the ESOP supplement the salary of an employee?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question is about employee benefits and salary, which seems to be a general topic that can be answered through a web search.
Action: router_tool
Action Input: {"question": "Does the ESOP supplement the salary of an employee?"} 

vectorstore

Final Answer: 'vectorstore'

> Finished chain.
 [DEBUG]: == [Router] Task output: 'vectorstore'


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question Does the ESOP suppleme

In [36]:
rag_crew = Crew(
    agents=[Router_Agent,Retriever_Agent,Grader_agent],
    tasks=[router_task,retriever_task,grader_task],
    verbose=True,

)
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question Does the ESOP supplement the salary of an employee?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question is about ESOP, salary, and employee, which seems to be related to finance and employment concepts.
Action: router_tool
Action Input: {"question": "Does the ESOP supplement the salary of an employee?"} 

vectorstore

Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question Does the ESOP supplement the salary of an e

In [37]:
rag_crew = Crew(
    agents=[Router_Agent,Retriever_Agent,Grader_agent,hallucination_grader],
    tasks=[router_task,retriever_task,grader_task,hallucination_task],
    verbose=True,

)
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question Does the ESOP supplement the salary of an employee?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question is about ESOP, which is a concept related to Retrieval-Augmented Generation, so I think it's eligible for a vectorstore search.
Action: router_tool
Action Input: {"question": "Does the ESOP supplement the salary of an employee?"} 

vectorstore

Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question Does the ESOP 

In [38]:
result

'no'

In [39]:
rag_crew = Crew(
    agents=[Router_Agent,Retriever_Agent,Grader_agent,hallucination_grader,answer_grader],
    tasks=[router_task,retriever_task,grader_task,hallucination_task,answer_task],
    verbose=True,

)
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question Does the ESOP supplement the salary of an employee?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question is about ESOP, which is a concept related to Retrieval-Augmented Generation, so I think it's eligible for a vectorstore search.
Action: router_tool
Action Input: {"question": "Does the ESOP supplement the salary of an employee?"} 

vectorstore

Final Answer: vectorstore

> Finished chain.
 [DEBUG]: == [Router] Task output: vectorstore


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question Does the ESOP 

In [40]:
result

'Yes, an Employee Stock Ownership Plan (ESOP) can supplement the salary of an employee by providing a retirement benefit or a supplemental income stream.'

In [41]:
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,

)
inputs = {"question":"What is Data Distributed Parallelism?"}
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question What is Data Distributed Parallelism?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question "What is Data Distributed Parallelism?" seems to be related to a technical concept, and the keywords do not specifically match any known concepts related to Retrieval-Augmented Generation.
Action: router_tool
Action Input: {"question": "What is Data Distributed Parallelism?"} 

web_search

Final Answer: websearch

> Finished chain.
 [DEBUG]: == [Router] Task output: websearch


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for

In [42]:
result

'Data Distributed Parallelism (DDP) is a technique used in parallel computing to divide a large dataset or a complex computation into smaller, independent tasks that can be executed simultaneously on multiple processing units, such as CPUs, GPUs, or clusters of nodes. This approach allows for significant speedup and scalability in processing large datasets or complex computations, making it a crucial technique in many fields, including scientific simulations, data analytics, and machine learning.'